In [1]:
import numpy as np
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import pandas as pd

On écrit une fonction pour créer la matrice

In [ ]:
def creation_matrice_ref():
    nb_mat_exterieur = 1000
    matrices_exterieur = [np.random.randn(n, n) for _ in range(nb_mat_exterieur)]
    mean_mat_exterieur = sum(matrices_exterieur) / nb_mat_exterieur
    mean_mat_exterieur[n//5:4*n//5, n//5:4*n//5] = 0

    nb_mat_milieu = 10000
    matrices_milieu = [np.random.randn(3*n//5, 3*n//5) for _ in range(nb_mat_milieu)]
    mean_mat_milieu = sum(matrices_milieu) / nb_mat_milieu
    mean_mat_milieu[n//5:2*n//5, n//5:2*n//5] = 0

    temp = np.zeros((n, n))
    temp[n//5:4*n//5, n//5:4*n//5] = mean_mat_milieu
    mean_mat_milieu = temp

    nb_mat_interieur = 100000
    matrices_interieur = [np.random.randn(n//5, n//5) for _ in range(nb_mat_interieur)]
    mean_mat_interieur = sum(matrices_interieur) / nb_mat_interieur

    temp = np.zeros((n, n))
    temp[2*n//5:3*n//5, 2*n//5:3*n//5] = mean_mat_interieur
    mean_mat_interieur = temp

    return mean_mat_exterieur + mean_mat_milieu + mean_mat_interieur


def creation_matrice_bruit(sd):
    nb_mat_exterieur = 1000
    matrices_exterieur = [
        np.random.randn(n, n) + np.random.normal(0, sd, (n, n))
        for _ in range(nb_mat_exterieur)
    ]
    mean_mat_exterieur = sum(matrices_exterieur) / nb_mat_exterieur
    mean_mat_exterieur[n//5:4*n//5, n//5:4*n//5] = 0

    nb_mat_milieu = 10000
    matrices_milieu = [
        np.random.randn(3*n//5, 3*n//5) + np.random.normal(0, sd, (3*n//5, 3*n//5))
        for _ in range(nb_mat_milieu)
    ]
    mean_mat_milieu = sum(matrices_milieu) / nb_mat_milieu
    mean_mat_milieu[n//5:2*n//5, n//5:2*n//5] = 0

    temp = np.zeros((n, n))
    temp[n//5:4*n//5, n//5:4*n//5] = mean_mat_milieu
    mean_mat_milieu = temp

    nb_mat_interieur = 100000
    matrices_interieur = [
        np.random.randn(n//5, n//5) + np.random.normal(0, sd, (n//5, n//5))
        for _ in range(nb_mat_interieur)
    ]
    mean_mat_interieur = sum(matrices_interieur) / nb_mat_interieur

    temp = np.zeros((n, n))
    temp[2*n//5:3*n//5, 2*n//5:3*n//5] = mean_mat_interieur
    mean_mat_interieur = temp

    return mean_mat_exterieur + mean_mat_milieu + mean_mat_interieur
